# 1. Setting up dependencies and config

In [ ]:
import os
import cv2
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt 

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import tensorflow as tf
from keras.models import Model
from keras.preprocessing import image
from keras.applications.xception import *
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout, Dense, GlobalAveragePooling2D

In [ ]:
# Defining configuration variables
labels = ['Black-grass',
          'Charlock',
          'Cleavers',
          'Common Chickweed',
          'Common wheat',
          'Fat Hen',
          'Loose Silky-bent',
          'Maize',
          'Scentless Mayweed',
          'Shepherds Purse',
          'Small-flowered Cranesbill',
          'Sugar beet']

# Enable/Disable data manipulation
GEN_DATA = False
UNZIP = False

# Version of the model being trained
DATA_VERSION = ""
SAVE_VERSION = "4"

# Defining data paths
DATA_DIR = 'drive/My Drive/plant-seedlings-classification'
TRAIN_SEG_PATH = f'{DATA_DIR}/train_seg{DATA_VERSION}'
TEST_SEG_PATH = f'{DATA_DIR}/test_seg{DATA_VERSION}'

# Path to store models and submissions
MODEL_PATH = f'{DATA_DIR}/Xception{SAVE_VERSION}.h5'
SUBMISSION_PATH = f'{DATA_DIR}/submission{SAVE_VERSION}.csv'

# Defining model hyperparameters
IMG_SIZE = 299
BATCH_SIZE = 16
TRAIN_IMG_COUNT = 4750
EPOCHS = 300
MODEL_COUNT=1

In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
# Unzipping the data if not already done
if UNZIP:
  ! unzip -uq "drive/MyDrive/plant-seedlings-classification/test.zip" -d "drive/MyDrive/plant-seedlings-classification/test"
  ! unzip -uq "drive/MyDrive/plant-seedlings-classification/train.zip" -d "drive/MyDrive/plant-seedlings-classification/train"

# 2. Segmenting the images

We iterate through all train and test set images and store their segmented versions

In [ ]:
# Defining a function to equalize the image's histogram
def equalize(img):
  img_YCrCb = cv2.cvtColor(img,cv2.COLOR_BGR2YCR_CB)
  channels = cv2.split(img_YCrCb)
  cv2.equalizeHist(channels[0],channels[0])
  cv2.merge(channels, img_YCrCb)
  cv2.cvtColor(img_YCrCb, cv2.COLOR_YCR_CB2BGR, img)
  return img

In [ ]:
# Defining a function to create a mask 
def create_mask(img):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    sensitivity = 35
    hsv_low = np.array([60 - sensitivity, 100, 50])
    hsv_high = np.array([60 + sensitivity, 255, 255])
    
    mask = cv2.inRange(img_hsv, hsv_low, hsv_high)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    return mask

In [ ]:
# Defining a function to segment an image
def segment(img):
    mask = create_mask(img)
    masked_img = cv2.bitwise_and(img, img, mask = mask)
    return masked_img

In [ ]:
# Defining a function to sharpen an image
def sharpen(img):
    img_blur = cv2.GaussianBlur(img, (0,0), 3)
    img_sharp = cv2.addWeighted(img, 1.5, img_blur, -0.5, 0)
    return img_sharp

In [ ]:
if GEN_DATA:
  # Creating the folder to save segmented training images
  if not os.path.isdir(TRAIN_SEG_PATH):
      os.mkdir(TRAIN_SEG_PATH)
      
  f, axarr = plt.subplots(1,2) # For showing a sample image

  # Segmenting the training data       
  for idx, label in enumerate(labels):
      
      folder = os.path.join(DATA_DIR, "train", label)
      
      show_img = True
      for img_name in os.listdir(folder):
          img_path = os.path.join(folder, img_name)
          img = cv2.imread(img_path, cv2.IMREAD_COLOR)
          img_eq = equalize(img)
          img_seg = segment(img_eq)
          img_sharp = sharpen(img_seg)
          seg_path = os.path.join(TRAIN_SEG_PATH, label)
          if not os.path.isdir(seg_path):
              os.mkdir(seg_path)
          cv2.imwrite(os.path.join(seg_path, img_name), img_sharp)
          if show_img: # Plotting a sample
              show_img = False
              axarr[0].imshow(img)
              axarr[1].imshow(img_seg)

In [ ]:
if GEN_DATA:
  # Creating the folder to save segmented testing images
  if not os.path.isdir(TEST_SEG_PATH):
      os.mkdir(TEST_SEG_PATH)
  
  # Segmenting the testing data  
  folder = os.path.join(DATA_DIR, "test")
  for img_name in os.listdir(folder):
      img_path = os.path.join(folder, img_name)
      img = cv2.imread(img_path, cv2.IMREAD_COLOR)
      img_eq = equalize(img)
      img_seg = segment(img_eq)
      img_sharp = sharpen(img_seg)
      seg_path = TEST_SEG_PATH
      cv2.imwrite(os.path.join(seg_path, img_name), img_sharp)

# 3. Training the model

In [ ]:
# Creating a data generator with data augmentation rules
# datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
#                              height_shift_range=0.3,
#                              horizontal_flip=True,
#                              rotation_range=180,
#                              vertical_flip=True,
#                              width_shift_range=0.3,
#                              zoom_range=0.3)

datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                             brightness_range=[0.5,1.5],
                             height_shift_range=0.3,
                             horizontal_flip=True,
                             rotation_range=180,
                             shear_range=0.2,
                             vertical_flip=True,
                             width_shift_range=0.3,
                             zoom_range=0.3)

In [ ]:
# For defining and compiling the models
def create_model():
  initial_model = Xception(weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False)
  x = initial_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dropout(0.5)(x)
  x = Dense(1024, activation='relu')(x)
  x = Dropout(0.5)(x)
  outputs = Dense(12, activation='softmax')(x)
  model = Model(inputs=initial_model.input, outputs=outputs)

  model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
# Creating the list of models
models = []

for i in range(MODEL_COUNT):
  models.append(create_model())

83689472/83683744 [==============================] - 1s 0us/step


In [ ]:
# Initializing the data generator
train_gen = datagen.flow_from_directory(TRAIN_SEG_PATH, 
                                       target_size=(IMG_SIZE, IMG_SIZE), 
                                       batch_size=BATCH_SIZE,
                                       class_mode='categorical')

Found 4750 images belonging to 12 classes.


In [ ]:
# Defining the learning rate scheduler
def scheduler(epoch, lr):
  print(lr)
  if epoch < 6:
    return lr
  else:
    # return lr * tf.math.exp(-0.1)
    return lr*0.9

In [ ]:
# Training the models
for i in range(MODEL_COUNT):
  curr_model_path = f'{DATA_DIR}/Xception{SAVE_VERSION}_{i}.h5'

  lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

  # Defining the checkpoint saving callback
  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(
      filepath=curr_model_path,
      save_weights_only=True,
      monitor='accuracy',
      mode='max',
      save_freq=TRAIN_IMG_COUNT//(10*BATCH_SIZE),
      save_best_only=True)

  print(f"=== Model number: {i+1} ===")
  models[i].fit_generator(train_gen, steps_per_epoch=TRAIN_IMG_COUNT//BATCH_SIZE, epochs=EPOCHS, callbacks=[lr_callback, ckpt_callback], verbose=1)

=== Model number: 1 ===


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/300
0.0010000000474974513
296/296 [==============================] - 1743s 6s/step - loss: 1.9901 - accuracy: 0.3359
Epoch 2/300
0.0010000000474974513
296/296 [==============================] - 190s 639ms/step - loss: 0.9573 - accuracy: 0.6811
Epoch 3/300
0.0010000000474974513
296/296 [==============================] - 179s 602ms/step - loss: 0.6821 - accuracy: 0.7751
Epoch 4/300
0.0010000000474974513
296/296 [==============================] - 179s 605ms/step - loss: 0.5498 - accuracy: 0.8112
Epoch 5/300
0.0010000000474974513
296/296 [==============================] - 179s 604ms/step - loss: 0.5651 - accuracy: 0.8263
Epoch 6/300
0.0010000000474974513
296/296 [==============================] - 182s 614ms/step - loss: 0.4774 - accuracy: 0.8403
Epoch 7/300
0.0010000000474974513
296/296 [==============================] - 181s 610ms/step - loss: 0.4170 - accuracy: 0.8695
Epoch 8/300
0.0009000000427477062
296/296 [==============================] - 179s 605ms/step - loss: 0.3570 - acc

# 4. Testing the model and saving the submission

In [ ]:
# Loading the model weights
models = []
for i in range(MODEL_COUNT-1):
  model = create_model()

  curr_model_path = f'{DATA_DIR}/Xception{SAVE_VERSION}_{i}.h5'
  model.load_weights(curr_model_path)
  models.append(model)

with open(SUBMISSION_PATH, 'w') as f:
  f.write('file,species\n')
  for file in tqdm(os.listdir(TEST_SEG_PATH)):
    img = image.load_img(os.path.join(TEST_SEG_PATH, file), target_size=(IMG_SIZE, IMG_SIZE))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    pred = np.zeros([12,])
    for model in models:
      for i, im in enumerate(datagen.flow(x)):
          pred += model.predict(im)[0]
          if i > 100:
              break
    f.write('{},{}\n'.format(file, labels[np.where(pred==np.max(pred))[0][0]]))

100%|██████████| 794/794 [2:47:02<00:00, 12.62s/it]
